In [1]:
# improt all libraries here so that you don't have to run this cell repeatedly
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

In [2]:
#data collection
# create an empty list that would be a bin for all our data dictionaries
list=[]
# this is to a round column as it is not present in the table we are scraping 
i = 0
# since we know the pattern of the urls let us loop though all of them
for i in range(1,9):
#     create an fstring so that the value of i change from 1 to 8
    url = f'https://web.enrboenyc.us/rcv/024306_{i}.html'
#     get the url, we don't need headers in this case because the website isn't asking for it
    response=requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
#     There is a simple way to scrape the table by using pd.read_html but since xlml doesn't work for me, I will scrape
# each element of the table
# SInce there are multiple table tages on the page let us use css selector to reach the one we are intersted in
    table=doc.select_one("#PanelReport > table > tbody > tr > td > table > tbody > tr:nth-child(11) > td > table").find_all('td')
#     Now let us loop through all the table to grab elemnts
    for row in table:
#         create an empty dictionary
        canDict = {}
#     now loop through hmtl tags
        for idx in row.find_all("tr"):
#         add name of candidate to the aforementioned dictionary
            canDict['name']= idx.find('td').text
    #add votes to the dictionary
            canDict['votes'] = idx.find('td').nextSibling.text
        # also add a column that would group the votes by rounds
            canDict['round'] = str(i)
#             send the dictionary to the empty list above
        list.append(canDict)
#    convert the list to a dataframe, drop nan values, reset index so that it doesn't break
voteData=pd.DataFrame(list).dropna().reset_index().drop(columns='index')

In [3]:
# create new columns
voteData['vote_count']=voteData.votes.str.split(n=1, expand=True)[0]
voteData['vote_share']=voteData.votes.str.split(n=1, expand=True)[1]

In [4]:
#remove unnecessary columns
cleaned_df = voteData[["round","name", "vote_count", "vote_share"]]

In [5]:
# repalce eliminated with 0 throughtout the dataset
cleaned_df=cleaned_df.replace("eliminated", "0")

In [8]:
# replace all symbols from the vote share columbs so that it can be coverted into a float
cleaned_df["vote_share"]=cleaned_df.vote_share.str.replace(
    "%", "").str.replace("(","").str.replace(")", "0").str.replace("with no choices left", "99999").astype(float)


In [9]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   round       120 non-null    object 
 1   name        120 non-null    object 
 2   vote_count  120 non-null    object 
 3   vote_share  71 non-null     float64
dtypes: float64(1), object(3)
memory usage: 3.9+ KB
